In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
import sklearn

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, precision_score, recall_score,  accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, precision_score, recall_score,  accuracy_score, precision_recall_curve

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as L
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [87]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/kgarg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kgarg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kgarg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/kgarg/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
!pip install emot

In [13]:
!pip install pyspellchecker
from spellchecker import SpellChecker

     |████████████████████████████████| 2.7 MB 1.8 MB/s eta 0:00:01


In [15]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import seaborn as sns
import matplotlib.pyplot as plt

import scipy

import warnings
warnings.filterwarnings('ignore')

import re
from collections import Counter
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [175]:
train = pd.read_csv('./data/Corona_NLP_train.csv', encoding="ISO-8859-1", low_memory=False)
test = pd.read_csv('./data/Corona_NLP_test.csv', encoding="ISO-8859-1", low_memory=False) 
df = train.append(test, sort = False)

In [176]:
# -- filling missing values
df['Location'].fillna(value='unknown', inplace=True) 

In [177]:
# -- label encoding 
encoding = {'Extremely Negative': 'Negative',
            'Extremely Positive': 'Positive'
           }
labels = ['Negative', 'Positive']
df['Sentiment'].replace(encoding, inplace=True)
df["sentiment"] = LabelEncoder().fit_transform(df["Sentiment"])

In [178]:
# -- remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

from bs4 import BeautifulSoup
# -- remove HTML
def html(text):
    return BeautifulSoup(text, "lxml").text

df['OriginalTweet'] = df['OriginalTweet'].apply(remove_urls)
df['OriginalTweet'] = df['OriginalTweet'].apply(html)

In [179]:
# -- lowercase
df['OriginalTweet'] = df['OriginalTweet'].str.lower()

In [180]:
# -- convert emojis and emoticons to words
def convert_emojis_emoticons(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

df['OriginalTweet'] = df['OriginalTweet'].apply(convert_emojis_emoticons)

In [181]:
# -- remove special and non-ASCII characters
def clean(tweet): 
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"â²", "", tweet)
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"â¹", "", tweet)
    tweet = re.sub(r"â½", "", tweet)
    tweet = re.sub(r"â¾", "", tweet)
    tweet = re.sub(r"ã¼berweist", "", tweet)
    tweet = re.sub(r"ã¼cretsiz", "", tweet)
    tweet = re.sub(r"zã¼rich", "", tweet)
    tweet = re.sub(r"ã¼retime", "", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    tweet = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'mentioned', tweet)
    tweet = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'referance', tweet)
    tweet = re.sub(r'£|\$', 'money', tweet)
    tweet = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', ' ', tweet)
    tweet = re.sub(r'\d+(\.\d+)?', ' ', tweet) 
    tweet = re.sub(r'[^\w\d\s]', ' ', tweet)
    tweet = re.sub(r'\s+', ' ', tweet)
    tweet = re.sub(r'^\s+|\s+?$', '', tweet.lower())
    tweet = re.sub(r"&gt;", ">", tweet)
    tweet = re.sub(r"&lt;", "<", tweet)
    tweet = re.sub(r"&amp;", "&", tweet) 
    tweet = re.sub(r"_", "  ", tweet)
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
    return str(tweet)

df['OriginalTweet'] = df['OriginalTweet'].apply(clean)

In [182]:
# -- spellcheck and keep words that are in the English dictionary
spell = SpellChecker(distance = 1, language='en')
words = set(nltk.corpus.words.words())

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else: 
            corrected_text.append(word)
    return " ".join(corrected_text)

def ensure_english(text):
    return " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())

df['OriginalTweet'] = df['OriginalTweet'].apply(correct_spellings)
df['OriginalTweet'] = df['OriginalTweet'].apply(ensure_english)

In [184]:
# -- remove punctuation
import string
regular_punct = list(string.punctuation)
def remove_punctuation(text):
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()

df['OriginalTweet'] = df['OriginalTweet'].apply(remove_punctuation)

In [186]:
# -- remove stopwords
en_stops = set(stopwords.words('english'))
def remove_stopwords(tweet):
  tweet = tweet.split()
  tweet = " ".join([word for word in tweet if not word in en_stops])  
  return tweet

df['OriginalTweet'] = df['OriginalTweet'].apply(remove_stopwords)

In [272]:
remove_stopwords('The current situation is not good')

'The current situation good'

In [188]:
# -- tokenize
df['OriginalTweet'] = df['OriginalTweet'].apply(word_tokenize)

In [189]:
# -- lematize
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
df['OriginalTweet'] = df['OriginalTweet'].apply(lemma_wordnet)

In [190]:
# -- combine individual words
def combine_text(input):
    combined = ' '.join(input)
    return combined
df['OriginalTweet'] = df['OriginalTweet'].apply(combine_text)

In [269]:
pd.options.display.max_colwidth=200
print(df['OriginalTweet'][201:210])

201                                       hearing multiple mandatory quarantine coming week u last chance stock food take warning late
202     stock amid outbreak latest extent damage share several key tech medium auto transportation travel retail consumer good banking
203                                                                  lot u going self isolation week thought useful article sane happy
204                                     panic today following impending ministry domestic trade consumer people spread fake news panic
205    increasingly global spread beginning panic grocery becoming increasingly chaotic animalistic nature survival starting hoard cut
206                                                         panic street food saffron flapjack nose food drink writer usual say recipe
207                                                                                            important update regarding retail store
208        pregnant make sure date wash frequently stay

In [192]:
# -- split into testing and testing set
train, test = train_test_split(df)

In [362]:
# -- Bag of words

cv = CountVectorizer()
X_train_bow = cv.fit_transform(train['OriginalTweet']) 
Y_train_bow = train['sentiment'] 
X_test_bow = cv.transform(test['OriginalTweet'])
Y_test_bow = test['sentiment']


In [351]:
# -- Bag of words - bigrams

cv = CountVectorizer(ngram_range=(1, 2))
X_train_bbow = cv.fit_transform(train['OriginalTweet']) 
X_test_bbow = cv.transform(test['OriginalTweet']) 
Y_train_bbow = train['sentiment']
Y_test_bbow = test['sentiment']

# -- TF-IDF

vectorizer = TfidfVectorizer(norm = None)
vectorizer.fit(train)
X_train_tfidf = vectorizer.fit_transform(train['OriginalTweet'])
X_test_tfidf = train['sentiment'] 
Y_train_tdidf =vectorizer.transform(test['OriginalTweet']) 


In [341]:
X_test_bbow.shape

(11239, 228214)

In [309]:
display("Bow-TF:IDF :", X_train_bbow.shape)
df_tfidf = pd.DataFrame(X_train_bbow.toarray(), columns=cv.get_feature_names())
display(df_tfidf.head())

'Bow-TF:IDF :'

(33716, 228214)

,aba,aba list,abandon,abandon also,abandon food,abandon need,abandon supermarket,abandoned,abandoned chilled,abandoned fate,...,zoom see,zoom shopping,zoom strategy,zoom toilet,zoom variety,zoom video,zoonotic,zoonotic deeply,zoonotic disease,zoonotic waiting
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [366]:
# -- multinomialNB with bag of word
from sklearn.naive_bayes import MultinomialNB
alpha = [1e-10, 1e-5, 0.1, 1.0, 2.0, 5.0]
for a in alpha:
    mnb = MultinomialNB(a)
    scores = sklearn.model_selection.cross_val_score(mnb, X_train_bbow, Y_train_bbow, cv=5)
    print('alpha: ', a)
    print(scores)
    print(np.mean(scores))
    print('\n')
clf = MultinomialNB()
clf.fit(X_train_bbow, Y_train_bbow)
clf.score(X_test_bbow, Y_test_bbow)

alpha:  1e-10
[0.60038553 0.59246626 0.60151268 0.58935192 0.5875723 ]
0.5942577373416024


alpha:  1e-05
[0.59311981 0.58371645 0.59587721 0.58341984 0.58416135]
0.5880589316384338


alpha:  0.1
[0.5569395  0.5571704  0.56102625 0.53937417 0.557467  ]
0.5543954637549524


alpha:  1.0
[0.65880783 0.65312176 0.65757081 0.64674477 0.65163874]
0.6535767816156256


alpha:  2.0
[0.64516607 0.64214741 0.64526175 0.63695684 0.63636364]
0.6411791438210426


alpha:  5.0
[0.62841044 0.62346137 0.62820703 0.62212665 0.62538929]
0.625518955644701




0.6551294599163626

In [285]:
# -- multinomialNB with TF-IDF
alpha = [175.0, 200.0, 225.0, 250.0, 300.0]
for a in alpha:
    mnb = MultinomialNB(a)
    scores = sklearn.model_selection.cross_val_score(mnb, X_train_tfidf, X_test_tfidf, cv=5)
    print('alpha: ', a)
    print(scores)
    print(np.mean(scores))
    print('\n')

alpha:  175.0
[0.63523132 0.63013495 0.63651194 0.62820703 0.62894854]
0.6318067557076457


alpha:  200.0
[0.63182088 0.62716892 0.63443571 0.62642741 0.62746552]
0.6294636861670526


alpha:  225.0
[0.628707   0.62553759 0.63206288 0.62183005 0.62524099]
0.6266757020021114


alpha:  250.0
[0.62692764 0.62286816 0.62909684 0.62019872 0.62449948]
0.6247181691342316


alpha:  300.0
[0.62396204 0.61752929 0.62375797 0.61663948 0.62079193]
0.6205361423093481




In [359]:
# -- decision tree classifier with BoW
dtclassifier=DecisionTreeClassifier(criterion="entropy", max_depth=None)
dtclassifier.fit(X_train_bow, Y_train_bow)
preddt = dtclassifier.predict(X_test_bow)

In [363]:
accuracy= accuracy_score(preddt,Y_test_bow)
print(accuracy)
# unigram: .69

0.6943678263190676


In [364]:
# -- decision tree classifier with tf-idf
dtclassifier=DecisionTreeClassifier(criterion="entropy", max_depth=None) 
dtclassifier.fit(X_train_tfidf,train['sentiment'])
preddt = dtclassifier.predict(Y_train_tdidf) 

In [365]:
accuracy= accuracy_score(preddt,Y_test)
print(accuracy)

0.6932111397811194


In [202]:
# -- decision tree classifier with hashing 
dtclassifier=DecisionTreeClassifier(criterion="entropy", max_depth=None)
dtclassifier.fit(X_train_hash,train['sentiment'])
preddt = dtclassifier.predict(Y_train_hash) 

In [203]:
accuracy= accuracy_score(preddt,Y_test)
print(accuracy)

0.641427173236053


In [204]:
# -- td-idf vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
vectors = vectorizer.fit_transform(df['OriginalTweet'])

In [205]:
test_vectors = vectors[40000:]
train_vectors = vectors[:40000]

In [206]:
# -- SVM, kernel=linear
import time
from sklearn import svm
from sklearn.metrics import classification_report
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, df['Sentiment'][:40000])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(df['Sentiment'][40000:], prediction_linear, output_dict=True)
print('positive: ', report['Positive'])
print('negative: ', report['Negative'])
print('notr: ', report['Neutral'])

Results for SVC(kernel=linear)
Training time: 192.519107s; Prediction time: 10.509840s
positive:  {'precision': 0.807565011820331, 'recall': 0.8275193798449613, 'f1-score': 0.8174204355108878, 'support': 2064}
negative:  {'precision': 0.8050153531218014, 'recall': 0.77145659637077, 'f1-score': 0.787878787878788, 'support': 2039}
notr:  {'precision': 0.6896162528216704, 'recall': 0.7171361502347418, 'f1-score': 0.7031070195627156, 'support': 852}


In [239]:
review = """give elderly disabled shopping amid outbreak"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['Neutral']


In [312]:
## -- logistic regression with hashing 
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs', multi_class="ovr")
log_reg.fit(X_train_bbow, X_test_bbow)

LogisticRegression(multi_class='ovr')

In [350]:
Y_train_bbow

<11239x228214 sparse matrix of type '<class 'numpy.int64'>'
	with 181700 stored elements in Compressed Sparse Row format>

In [355]:
train_accuracy = log_reg.score(X_train_bbow, Y_train_bbow)
test_accuracy = log_reg.score(X_test_bbow, Y_test_bbow)
test_accuracy

0.7882373876679419

In [224]:
log_reg_mnm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg_mnm.fit(X_train_hash, X_test_hash)

train_accuracy = log_reg_mnm.score(X_train_hash, X_test_hash)
test_accuracy = log_reg_mnm.score(Y_train_hash, Y_test)

print('Multinomial (Softmax)', '-'*20, 
      'Accuracy on Train Data : {:.2f}'.format(train_accuracy), 
      'Accuracy on Test Data  : {:.2f}'.format(test_accuracy), sep='\n')

Multinomial (Softmax)
--------------------
Accuracy on Train Data : 0.83
Accuracy on Test Data  : 0.78


In [225]:
# -- validation for multinomial log reg 
C_values = [0.001,0.01, 0.1,1,10,100, 1000]

accuracy_values = pd.DataFrame(columns=['C_values', 'Train Accuracy', 'Test Accuracy'])

for c in C_values:
    # Apply logistic regression model to training data
    lr = LogisticRegression(penalty = 'l2', C = c, random_state = 0, solver='lbfgs', multi_class='multinomial')
    lr.fit(X_train_hash, X_test_hash)
    accuracy_values = accuracy_values.append({'C_values': c,
                                              'Train Accuracy': lr.score(X_train_hash, X_test_hash),
                                              'Test Accuracy': lr.score(Y_train_hash, Y_test)
                                             }, ignore_index=True)
display(accuracy_values)    

,C_values,Train Accuracy,Test Accuracy
0,0.001,0.476361,0.479224
1,0.010,0.590728,0.584216
2,0.100,0.719213,0.697482
3,1.000,0.829962,0.775514
4,10.000,0.868668,0.783522
5,100.000,0.874184,0.774980
6,1000.000,0.874659,0.773200


In [ ]:
# -- grid search
parameters = {"C": [10 ** x for x in range (-5, 5, 1)],
              "penalty": ['l1', 'l2']
             }

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_cv = GridSearchCV(estimator=log_reg,
                       param_grid = parameters,
                       cv = 10
                      )

grid_cv.fit(X_train_hash, X_test_hash)

GridSearchCV(cv=10, estimator=LogisticRegression(multi_class='ovr'),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l1', 'l2']})

In [ ]:
print("Best Parameters : ", grid_cv.best_params_)
print("Best Score      : ", grid_cv.best_score_)

Best Parameters :  {'C': 10, 'penalty': 'l2'}
Best Score      :  0.8185134316740406


In [ ]:

%time results = grid_cv.cv_results_

df1 = pd.DataFrame(results)
display(df1.head(35))
df1.info()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.2 µs


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002367,0.000275,0.000000,0.000000,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
1,3.313961,0.044434,0.010587,0.000077,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.437129,0.436833,0.437129,0.437129,0.437129,0.437129,0.437259,0.437259,0.437259,0.437259,0.437152,0.000123,9
2,0.002228,0.000192,0.000000,0.000000,0.0001,l1,"{'C': 0.0001, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
3,3.190604,0.081269,0.011091,0.001986,0.0001,l2,"{'C': 0.0001, 'penalty': 'l2'}",0.437129,0.436833,0.437129,0.437129,0.437129,0.437129,0.437259,0.437259,0.437259,0.437259,0.437152,0.000123,9
4,0.002420,0.000151,0.000000,0.000000,0.001,l1,"{'C': 0.001, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
5,3.347032,0.042141,0.011306,0.002212,0.001,l2,"{'C': 0.001, 'penalty': 'l2'}",0.438019,0.439798,0.438909,0.438019,0.438909,0.439502,0.439632,0.439929,0.439039,0.439632,0.439139,0.000656,8
6,0.002300,0.000184,0.000000,0.000000,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
7,6.741687,0.069908,0.011410,0.002420,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.575030,0.559312,0.567023,0.551008,0.570285,0.564947,0.557995,0.563631,0.564818,0.555918,0.562997,0.006738,7
8,0.002367,0.000176,0.000000,0.000000,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
9,17.001108,0.650369,0.011164,0.001499,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",0.684164,0.673488,0.678826,0.653025,0.682681,0.668743,0.674281,0.680510,0.673984,0.659745,0.672944,0.009510,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mean_fit_time      20 non-null     float64
 1   std_fit_time       20 non-null     float64
 2   mean_score_time    20 non-null     float64
 3   std_score_time     20 non-null     float64
 4   param_C            20 non-null     object 
 5   param_penalty      20 non-null     object 
 6   params             20 non-null     object 
 7   split0_test_score  10 non-null     float64
 8   split1_test_score  10 non-null     float64
 9   split2_test_score  10 non-null     float64
 10  split3_test_score  10 non-null     float64
 11  split4_test_score  10 non-null     float64
 12  split5_test_score  10 non-null     float64
 13  split6_test_score  10 non-null     float64
 14  split7_test_score  10 non-null     float64
 15  split8_test_score  10 non-null     float64
 16  split9_test_score  10 non-nu

In [ ]:
df1 = df1[['param_penalty','param_C', 'mean_test_score']]
df1 = df1.sort_values(by='mean_test_score', ascending = False)
df1

,param_penalty,param_C,mean_test_score
13,l2,10,0.818513
15,l2,100,0.812345
17,l2,1000,0.800036
19,l2,10000,0.795409
11,l2,1,0.774321
9,l2,0.1,0.672944
7,l2,0.01,0.562997
5,l2,0.001,0.439139
1,l2,1e-05,0.437152
3,l2,0.0001,0.437152
